<a href="https://colab.research.google.com/github/jsgovea/PyTorch-learning/blob/main/cnn_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, sampler
import torchvision.datasets as datasets
import torchvision.transforms as T
import matplotlib.pyplot as plt


In [9]:
DATA_PATH = '/content/cifar-10'
NUM_TRAIN = 50000
NUM_VAL = 5000
NUM_TEST = 5000
MINBATCH_SIZE = 64

transform_cifar = T.Compose([
    T.ToTensor(),
    T.Normalize([0.491, 0.482, 0.447], [0.247, 0.243, 0.261])
])

In [12]:
# Train dataset

# Download the dataset
cifar10_train = datasets.CIFAR10(DATA_PATH, train=True, download=True,
                                 transform=transform_cifar)

train_loader = DataLoader(cifar10_train, batch_size=MINBATCH_SIZE,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = datasets.CIFAR10(DATA_PATH, train=False, download=True,
                              transform=transform_cifar)

val_loader = DataLoader(cifar10_val, batch_size=MINBATCH_SIZE,
                        sampler=sampler.SubsetRandomSampler(range(NUM_VAL)))

cifar10_test = datasets.CIFAR10(DATA_PATH, train=False, download=True,
                                transform=transform_cifar)

test_loader = DataLoader(cifar10_test, batch_size=MINBATCH_SIZE,
                         sampler=sampler.SubsetRandomSampler(range(NUM_VAL, len(cifar10_test))))



Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [15]:
test_loader.dataset

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: /content/cifar-10
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.261])
           )

## Check the GPUs

In [17]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(device)

cpu


## Show images

In [ ]:
classes = test_loader.dataset.classes
def plot_figure(image):
  plt.imshow(np.transpose(image(1, 2, 0)))
  plt.axis('off')
  plt.show()

## Calculate accuracy